# A Brief Introduction to AuthoritySpoke

> "Details, unnumbered, shifting, sharp, disordered, unchartable, jagged."
>
> Llewellyn, Karl N. _The Bramble Bush: On Our Law and Its Study_ 106 (Quid Pro 2012) (1930).
    
AuthoritySpoke is a Python package for making legal analysis readable by both computers and humans.

This notebook will provide an overview of AuthoritySpoke's most important features. Please remember that AuthoritySpoke is in an early alpha state, so many features have yet to be implemented, and some others have disappointing limits to their functionality.

## 1. Importing the Package

Let's start by importing the package and its Opinion class. 

(When I capitalize a seemingly random word, it's probably because it's the name of an AuthoritySpoke class.)

In [1]:
import authorityspoke
from authorityspoke import Opinion

If you executed that cell with no error messages, then it worked! 

If you got a message like `No module named 'authorityspoke'`, then AuthoritySpoke is probably not installed in your current Python environment. In that case, check the documentation for help.

## 2. Downloading Opinions

Now we need some court opinions to load into AuthoritySpoke. We can collect these from the Case Access Project API, a project of the Harvard Law School Library Innovation Lab. To download full cases from CAP, you'll need to [register for an API key](https://case.law/user/register/). However, if you'd rather skip forward to the end of this section without interacting with the API, you can. There's already a copy of the files we're going to download in the example_data/opinions folder of this repository.

The CAP API limits users to downloading 500 full cases per day. If you accidentally make a query that returns hundreds of full cases, you could hit your limit before you know it. You should first try out your API queries without the "full_case": "true" parameter, and then only request full cases once you're confident you'll receive what you expect. To minimize the risk of revealing your API key to others, you can store it in an environment variable called CAP_API_KEY and then retrieve it as shown in the cell below.

In [2]:
import os

CAP_API_KEY = os.environ['CAP_API_KEY']

Next we need to choose which cases to download for our analysis.

Let's download Oracle America v. Google (2014), a landmark opinion in which the Federal Circuit Court of Appeals held that the API declarations for the Java language were copyrightable. And since we'll want to compare the Oracle case to a related case, let's also download Lotus Development Corporation v. Borland International (1995). In that case, the First Circuit Court of Appeals held that the menu structure of a spreadsheet program called Lotus 1-2-3 was uncopyrightable because it was a "method of operation" under the Copyright Act. As we'll see, the Oracle case discusses and disagrees with the Lotus case.

Citations
* Oracle America v. Google: 750 F.3d 1339
* Lotus Development Corporation v. Borland International: 49 F.3d 807

In [3]:
oracle = Opinion.cap_download(cite="750 F.3d 1339", filename="oracle_h.json")

Now we have a record representing the Oracle case, which has been saved to the example_data/opinions folder under the filename "oracle_h.json". (I included the "h" in the filename to remind me that this file came from the Harvard CAP API.) The data from the API has also been used to generate an AuthoritySpoke Opinion object assigned to the variable name "oracle", so let's look at it.

In [4]:
oracle

Opinion(name='ORACLE AMERICA, INC., Plaintiff-Appellant, v. GOOGLE INC., Defendant-Cross-Appellant', name_abbreviation='Oracle America, Inc. v. Google Inc.', citations=('750 F.3d 1339',), first_page=1339, last_page=1381, decision_date=datetime.date(2014, 5, 9), court='fed-cir', position='majority', author=None)

Yes, this is the case I expected. But if I had provided my API key and used the full_case flag, I could have received more information, like whether there are any non-majority opinions in the case, and the names of the opinion authors. So let's request both the Oracle and Lotus opinions with "full_case": "true".

In [5]:
oracle = Opinion.cap_download(
    cite="750 F.3d 1339", 
    filename="oracle_h.json", 
    full_case=True, 
    api_key=CAP_API_KEY)

In [6]:
oracle

Opinion(name='ORACLE AMERICA, INC., Plaintiff-Appellant, v. GOOGLE INC., Defendant-Cross-Appellant', name_abbreviation='Oracle America, Inc. v. Google Inc.', citations=('750 F.3d 1339',), first_page=1339, last_page=1381, decision_date=datetime.date(2014, 5, 9), court='fed-cir', position='majority', author='O’MALLEY, Circuit Judge.')

In [7]:
lotus = Opinion.cap_download(
    cite="49 F.3d 807", 
    filename="lotus_h.json", 
    full_case=True, 
    api_key=CAP_API_KEY)

In [8]:
lotus

[Opinion(name='LOTUS DEVELOPMENT CORPORATION, Plaintiff, Appellee, v. BORLAND INTERNATIONAL, INC., Defendant, Appellant', name_abbreviation='Lotus Development Corp. v. Borland International, Inc.', citations=('49 F.3d 807',), first_page=807, last_page=822, decision_date=datetime.date(1995, 3, 9), court='1st-cir', position='majority', author='STAHL, Circuit Judge.'),
 Opinion(name='LOTUS DEVELOPMENT CORPORATION, Plaintiff, Appellee, v. BORLAND INTERNATIONAL, INC., Defendant, Appellant', name_abbreviation='Lotus Development Corp. v. Borland International, Inc.', citations=('49 F.3d 807',), first_page=807, last_page=822, decision_date=datetime.date(1995, 3, 9), court='1st-cir', position='concurrence', author='BOUDIN, Circuit Judge')]

When we queried the API for the Lotus case, we discovered that it had two opinions, a majority and a concurrence. Let's just work with the majority opinion.

In [9]:
lotus_majority = lotus[0]

Finally, what should you do if you chose not to get an API key or were unable to create the Opinion objects from downloaded data? Use the following commands to create the Opinion objects from the files in the example_data/opinions folder.

If you already did the steps above, you can skip the next cell and go to section 3.

In [10]:
oracle = Opinion.lead_opinion_from_file("oracle_h.json")
lotus_majority = Opinion.lead_opinion_from_file("lotus_h.json")

## 3. Expositing the Opinion

Now we can link some legal analysis to each opinion by using its `exposit` method. The parameter we pass to this function is the name of a JSON file containing structured information about the legal holdings posited by the opinion. Sadly, the `exposit` method requires you to exposit the Opinion for the computer's benefit, not the other way around.

A terminology note: When an Opinion asserts that a Rule is valid law, the Opinion **posits** the Rule and makes the Rule one of the Opinion's **holdings**. When you, as a reader, assert that an Opinion posited a Rule, you **exposit** the Opinion.

Anyway, the example JSON files have already been created and AuthoritySpoke should find them when we call the `exposit` method.

In [11]:
oracle = oracle.exposit("holding_oracle.json")
lotus_majority = lotus_majority.exposit("holding_lotus.json")

Before we see what that did, let's look at the format of the holding JSON files we just loaded in.

The top level of each of these JSON files has two fields: `"mentioned_factors"` and `"holdings"`. `"mentioned_factors"` is where you list the people, places, and things that need to be mentioned more than once to explain a holding. The reason you're listing these entities is so AuthoritySpoke will recognize that the same entities have been mentioned more than once in different contexts. Each of these entities has a `"type"` attribute, which will be the Python class name for the object that's created. You can use the generic type "entity", or the more specific subtypes "association", "human", or "event".

Here's the `"mentioned_factors"` field from holding_oracle.json.

```
"mentioned_factors": [

        {
            "type": "association",
            "name": "Oracle America"
        },
        {
            "type": "association",
            "name": "Google"
        },
        {
            "type": "association",
            "name": "Sun Microsystems"
        },
        {
            "type": "entity",
            "name": "the Java API"
        },
        {
            "type": "entity",
            "name": "the Java language"
        }
    ]
```

holding_oracle.json also contains a list of holdings. These are legal rules supported by the text of the Oracle opinion. Let's look at the part of holding_oracle.json that represents a single holding.

Currently, AuthoritySpoke has a limited proof-of-concept function for linking Rules to the enactment text that they cite (e.g. statutes, constitutional provisions, and regulations), but AuthoritySpoke doesn't yet have the ability to link holdings to specific passages within an Opinion. In fact, it doesn't yet display the text of opinions at all. But the first holding in holding_oracle.json is derived from the following text in Oracle America v. Google:

> By statute, a work must be “original” to qualify for copyright protection. 17 U.S.C. § 102(a).

And that rule is represented in the JSON like this:

```
"holdings": [{
        "inputs": {
            "type": "fact",
            "content": "the Java API was an original work",
            "truth": false
        },
        "outputs": {
            "type": "fact",
            "content": "the Java API was copyrightable",
            "truth": false
        },
        "mandatory": true,
        "enactments": {
            "code": "usc17.xml",
            "section": "102",
            "subsection": "a",
            "end": "machine or device.",
            "name": "copyright protection provision"
        }
    },
    ...
    ]
```
This JSON means that according to the cited enactment, if it's false that "the Java API was an original work", then it's mandatory for the court to find it to be false that "the Java API was copyrightable".

As you can see, Rules in AuthoritySpoke can have `"inputs"` and `"outputs"`. Inputs are the preconditions for applying the Rule, and outputs are the results. Not shown here, Rules can also have `"despite"` Factors, which are Factors that don't need to be present to trigger the rule, but that don't prevent the rule from applying if they're present. There can be more than one Factor in the `"inputs"`, `"outputs"` or `"despite"` categories, and if so they would be listed together in square brackets in the JSON.

The JSON representation of a Rule can also have `"mandatory"` and `"universal"` fields. If omitted, the values of these fields are implied as False. `"universal"` means that the Rule applies whenever its inputs are present. `"mandatory"` means that when Rule applies, the court has no discretion and must accept the outputs.

The JSON can also contain fields representing Enactments. It identifies a passage of legislative text by naming the code where the passage can be found (which currently has to be a file in the example_data/codes directory) and then specifying a section and subsection. If the relevant passage isn't an entire section or subsection, `"start"` and `"end"` phrases can identify where the passage begins or ends. The `"name"` field is simply a nickname that can be used to refer to the passage again later in the same file.

There can also be an `"enactments_despite"` field, which identifies legislative text that doesn't need to be present for the Rule to apply, but that also doesn't negate the validity of the Rule.

## 4. Viewing an Opinion's Holdings

If you take a look in holding_oracle.json, you'll see that there are 20 holdings. (You can verify this by checking how many times the string "inputs" appears in the file.)

Lets make sure that the .exposit() method added all of those holdings to our oracle Opinion object.

In [12]:
len(oracle.holdings)

20

Now let's see the string representation of the AuthoritySpoke Rule object we created from the structured JSON we saw above.

In [13]:
print(oracle.holdings[0])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact it is false that <the Java API> was copyrightable
based on the input
GIVEN: fact it is false that <the Java API> was an original work
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)



I'd like to find ways to make this format easier to read, but for now let's try to pick it apart.

Instead of the terms "input" and "output" we saw in the JSON file, we now have "GIVEN" and "RESULT". And the "RESULT" comes first, because it's hard to understand anything else about a legal rule until you understand what it does. Also, notice that the word "GIVEN" also precedes the statute citation. Was the reuse of the word "GIVEN" useful to remind you that the existence of the statutory text is one of the preconditions for applying the Rule? Or did you just find it confusing?

It's also important to notice that a Rule can be purely hypothetical from the point of view of the Opinion that posits it. In this case, the court finds that there would be a certain legal significance if it was "GIVEN" that `it is false that <the Java API> was an original work`, but the court isn't going to find that precondition applies, so it's also not going to accept the result `it is false that <the Java API> was copyrightable`.


We can also access just the inputs of a Rule, just the Enactments, etc.

In [14]:
oracle.holdings[0].inputs

(Fact(predicate=Predicate(content='{} was an original work', truth=False, reciprocal=False, comparison=None, quantity=None), context_factors=(Entity(name='the Java API', generic=True, plural=False),), name=None, standard_of_proof=None, absent=False, generic=False, case_factors=()),)

In [15]:
oracle.holdings[0].enactments

(Enactment(code=<authorityspoke.enactments.Code object at 0x113424828>, section='102', subsection='a', start=None, end='machine or device.', name='copyright protection provision'),)

## 5. Generic Factors

The two instances of the phrase "the Java API" are in angle brackets because that phrase is one of the `"mentioned_factors"` we defined in the JSON. More specifically, the angle brackets are there because "Entity" objects are considered "generic" by default, and we didn't specify otherwise.

In [16]:
oracle.holdings[0].generic_factors

[Entity(name='the Java API', generic=True, plural=False)]

A generic Entity is "generic" in the sense that in the context of the Factor or Rule where the Entity appears, it could be replaced with some other Entity without changing the meaning.

To imagine applying the same rule in a different copyright case, let's try creating a new Rule that's the same as the original, except instead of using 'the Java API' as its generic Entity, we'll use 'Nosferatu'.

In [17]:
from authorityspoke import Entity

nosferatu_rule = oracle.holdings[0].new_context(
    {Entity('the Java API'): Entity('Nosferatu')}
)

In AuthoritySpoke, Rule and Factor objects are "frozen" objects, which means Python will try to prevent you from modifying the object after it has been created. The `new_context` method returns a new Rule object, which we've assigned to the name `nosferatu_rule`, but the Rule that we used as a basis for the new object also still exists, and it's unchanged.

In [18]:
print(oracle.holdings[0])
print(nosferatu_rule)

the rule that the court MUST SOMETIMES accept the result
RESULT: fact it is false that <the Java API> was copyrightable
based on the input
GIVEN: fact it is false that <the Java API> was an original work
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)

the rule that the court MUST SOMETIMES accept the result
RESULT: fact it is false that <Nosferatu> was copyrightable
based on the input
GIVEN: fact it is false that <Nosferatu> was an original work
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, rep

Even though these Rules have different generic Factors, they evaluate equal to one another because they have the same meaning.

In [19]:
oracle.holdings[0] == nosferatu_rule

True

## 6. Enactment Objects and Implication

Let's look at the attributes of the Enactment object for this Rule.

In [20]:
copyright_provision = oracle.holdings[0].enactments[0]
copyright_provision

Enactment(code=<authorityspoke.enactments.Code object at 0x113424828>, section='102', subsection='a', start=None, end='machine or device.', name='copyright protection provision')

The Enactment object refers to a Code object, which is an instance of an AuthoritySpoke class representing a code of laws.

In [21]:
usc = copyright_provision.code
print(usc)

USC Title 17


Let's try creating a new Enactment object representing a shorter passage of text from the same Code.

In [22]:
from authorityspoke import Enactment

works_of_authorship_clause = Enactment(
                code=usc,
                section=102,
                subsection="a",
                end="works of authorship"
)

Now we can create a new Rule object that cites to our new Enactment object rather than the old one. This time, instead of using the `new_context` method to create a new Rule object, we'll use the `evolve` method. With the `evolve` method, instead of specifying Factors that should be replaced wherever they're found, we specify which attributes from the Rule object's `__init__` method we want to replace, and then specify what we want to replace those attributes' old values with.

In [23]:
rule_with_shorter_enactment = oracle.holdings[0].evolve(
            {"enactments": works_of_authorship_clause}
        )

In [24]:
print(rule_with_shorter_enactment)

the rule that the court MUST SOMETIMES accept the result
RESULT: fact it is false that <the Java API> was copyrightable
based on the input
GIVEN: fact it is false that <the Java API> was an original work
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship" (USC Title 17, 102)



Now let's try comparing this new Rule with the real Rule from the Oracle case, to see whether one implies the other. When you're comparing AuthoritySpoke objects, the greater than sign `>` means "implies, but is not equal to".

In [25]:
rule_with_shorter_enactment > oracle.holdings[0]

True

You can also use the greater than or equal sign `>=` to mean "implies or is equal to". In logic, it's common to say that identical statements also imply one another, so that would mean `>=` is the symbol that really means "implies".

In [26]:
oracle.holdings[0] >= rule_with_shorter_enactment

False

By comparing the string representations of the original Rule from the Oracle case and `rule_with_shorter_enactment`, can you tell why the latter implies the former, and not the other way around?

If you guessed that it was because `rule_with_shorter_enactment` has a shorter Enactment, you're right. Rules that require fewer, or less specific, inputs are _broader_ than Rules that have more inputs, because there's a larger set of situations where those Rules can be triggered.

If this relationship isn't clear to you, imagine some "Enactment A" containing only a subset of the text of "Enactment B", and then imagine what would happen if a legislature amended some of the statutory text that was part of Enactment B but not of Enactment A. A requirement to cite Enactment B would no longer be possible to satisfy, because some of that text would no longer be available. Thus a requirement to cite Enactment A could be satisfied in every situation where a requirement to cite Enactment B could be satisfied, and then some.

## 7. Contradictions Between Rules

So far we've only looked at one real holding, so let's look at a significant holding from the Lotus case.

It says that under a statute providing that "In no case does copyright protection for an original work of authorship extend to any...method of operation", the fact that a Lotus menu command hierarchy was a "method of operation" meant that it was also uncopyrightable, despite a couple of Facts that might tempt some courts to rule the other way.

In [27]:
print(lotus_majority.holdings[8])

the rule that the court MUST ALWAYS accept the result
RESULT: fact it is false that <the Lotus menu command hierarchy> was copyrightable
based on the input
GIVEN: fact <the Lotus menu command hierarchy> was a method of operation
DESPITE: fact a text described <the Lotus menu command hierarchy>
DESPITE: fact <the Lotus menu command hierarchy> was an original work
according to the legislation
GIVEN: "In no case does copyright protection for an original work of authorship extend to any" (USC Title 17, 102)
GIVEN: "method of operation" (USC Title 17, 102)



Lotus was a case relied upon by Google in the Oracle v. Google case, but Oracle was the winner in that decision. So we might wonder whether the Oracle Opinion contradicts the Lotus Opinion. Let's check.

In [28]:
oracle.contradicts(lotus_majority)

True

Good to know! But maybe we want more detail than that. Let's check each Rule posited by the Oracle case to see whether it contradicts lotus_majority.holdings[8].

In [29]:
for index, oracle_rule in enumerate(oracle.holdings):
    print(f'{index:02} {oracle_rule.contradicts(lotus_majority.holdings[8])}')

00 False
01 False
02 False
03 False
04 False
05 False
06 False
07 False
08 False
09 False
10 True
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False


It looks like the Rule at index 10 of oracle.holdings contradicts the Lotus court's Rule.
Let's read it and see if we can spot the contradiction.

In [30]:
print(oracle.holdings[10])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact <the Java API> was copyrightable
based on the input
GIVEN: fact <the Java language> was a computer program
GIVEN: fact <the Java API> was a set of application programming interface declarations
GIVEN: fact <the Java API> was an original work
GIVEN: fact <the Java API> was a non-literal element of <the Java language>
GIVEN: fact <the Java API> was the expression of an idea
GIVEN: fact it is false that <the Java API> was essentially the only way to express the idea that it embodied
GIVEN: fact <the Java API> was creative
GIVEN: fact it was possible to use <the Java language> without copying <the Java API>
DESPITE: fact <the Java API> was a method of operation
DESPITE: fact <the Java API> contained short phrases
DESPITE: fact <the Java API> became so popular that it was the industry standard
DESPITE: fact there was a preexisting community of programmers accustomed to using <the Java API>
according to the legislation
GIV

That's quite a bit of detail. It's good that our computers are going to be able to read these things for us!

The first obvious contrast between `lotus_majority.holdings[8]` and `oracle.holdings[10]` is that the Rule from the Lotus case is relatively succinct and categorical. The Lotus court interprets Section 102(b) of the Copyright Act to mean that if a work is a "method of operation", it's simply impossible for that work to be copyrighted, so it's not necessary to consider a lot of case-specific facts to reach a conclusion.

The Federal Circuit's Oracle decision complicates that view significantly. The Federal Circuit believes that the fact that an API is, or hypothetically might be, a "method of operation" is only one of many factors that a court can consider in deciding copyrightability. The following quotation, repeated in the Oracle case, illustrates the Federal Circuit's view.

> “Section 102(b) does not extinguish the protection accorded a particular expression of an idea merely because that expression is embodied in a method of operation.” Mitel, Inc. v. Iqtel, Inc., 124 F.3d 1366, 1372 (10th Cir.1997)

And that's why AuthoritySpoke finds a contradiction between these two Rules. The Oracle opinion says that courts can sometimes accept the result `fact <the Java API> was copyrightable` despite the fact `<the Java API> was a method of operation`. The Lotus Opinion would consider that impossible.

By the way, AuthoritySpoke isn't applying any fancy grammatical parsing to understand the meaning of each Factor. AuthoritySpoke won't recognize that Factors' meanings are related in any way unless their `content` values are exactly the same string, with the exception of references to generic entities. (And also an optional numeric value that can come at the end of the string, which isn't demonstrated in this notebook.)

## 8. Nuances of Meaning in Rules

Let's look at one more sentence from the Oracle Opinion, so I can point out a few more design decisions AuthoritySpoke makes in representing procedural Rules.

> In the Ninth Circuit, while questions regarding originality are considered questions of copyrightability, concepts of merger and scenes a faire are affirmative defenses to claims of infringement.

(The "merger" doctrine says that a work is deemed to be "merged" with an uncopyrightable idea if it's essentially the only way to express the idea. "Scenes a faire" is a concept applied mostly to works of fiction, and it means that conventional genre tropes are not copyrightable.)

The quoted sentence is fairly ordinary, as court opinions go, but I found several interesting challenges in creating structered data about its procedural meaning.

1. The sentence describes what the law is "In the Ninth Circuit". You might remember that the court that issued the Oracle opinion was the Federal Circuit, not the Ninth Circuit. So the Federal Circuit is deciding what it thinks that the Ninth Circuit thinks that Congress meant by enacting the statute. The middle layer of this interpretation, in which the Federal Circuit attributes a belief to the Ninth Circuit, is simply absent from the AuthoritySpoke model of the Rule. However, future updates to AuthoritySpoke might make it possible to capture this information.

2. The sentence uses the concept of an "affirmative defense", which generally means a defense that the defendant has the burden of proof to establish. I chose to model this concept by writing that if one of the facts that would establish the affirmative defense is present, then it could be established that the copyright was not infringed, but if they are both absent, then the copyright could have been infringed. I'm sure some legal experts would find this too simplistic, and would argue that it's not possible to formalize the concept of an affirmative defense without explicitly mentioning procedural concepts like a burden of proof.

3. The sentence seems to have something to say about what happens if either of two Factors are present, or if both of them are absent. That makes three different Rules. It's not ideal for one sentence to explode into multiple different Python objects when it's formalized, and it's worth wondering whether there would have been a way to pack all the information into a single object. 

4. I noticed that the concept of a copyrighted work being "merged" or being a "scene a faire" are both characteristics intrinsic in the copyrighted work, and don't depend on the characteristics of the allegedly infringing work. So if a work that's "merged" or is a "scene a faire" can't be infringed, that means there are some works that are "copyrightable" but that can never be infringed by any other work. I suspect that the court's interpretation of these legal categories could confuse future courts and parties, with the result that the "merger" or "scene a faire" concepts could fall through the cracks and be ignored. Would there be a useful way to have AuthoritySpoke flag the issue that a conclusion about two different Entities is being based on Factors that only relate to one of those two Entities?

The three Rule objects used to represent the sentence from the Oracle opinion can be found in the Appendix below. They're `oracle.holdings[11]` through `oracle.holdings[13]`.

## Appendix: Rule Objects in *Oracle v. Google* and *Lotus v. Borland*

This Appendix will list all of the Rule objects in `oracle.holdings` and `lotus_majority.holdings`. Each Rule will be preceded by a passage from the Opinion that indicates the Opinion has endorsed the Rule. Eventually, I'd like to add a command to link Rules to Opinion text within AuthoritySpoke itself, but that's currently not available.

To find the full text of the opinions, look in the example_data/opinions/ folder. The text delivered by the CAP API was collected from print sources, so it will contain some Optical Character Recognition errors.

### *Lotus v. Borland* 49 F.3d 807 (1995)

> To establish copyright infringement, a plaintiff must prove "(1) ownership of a valid copyright, and (2) copying of constituent elements of the work that are original."

In [31]:
print(lotus_majority.holdings[0])
print(lotus_majority.holdings[1])
print(lotus_majority.holdings[2])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact <Borland International> infringed the copyright in <the Lotus menu command hierarchy>
based on the input
GIVEN: fact <the Lotus menu command hierarchy> was copyrightable
GIVEN: fact <Borland International> copied constituent elements of <the Lotus menu command hierarchy> that were original
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)

the rule that the court MAY ALWAYS accept the result
RESULT: absence of fact <Borland International> infringed the copyright in <the Lotus menu command hierarchy>
based on the input
GIVEN: absence of fact <the Lotus menu command hierarchy> was copyrightable
according to the legislati

> In judicial proceedings, a certificate of copyright registration constitutes prima facie evidence of copyrightability and shifts the burden to the defendant to demonstrate why the copyright is not valid.

In [32]:
print(lotus_majority.holdings[4])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact <the Lotus menu command hierarchy> was copyrightable
based on the input
GIVEN: evidence of certificate of copyright registration, which supports fact <Lotus Development Corporation> registered a copyright covering <the Lotus menu command hierarchy>
GIVEN: absence of fact it is false that <the Lotus menu command hierarchy> was copyrightable
according to the legislation
GIVEN: "In any judicial proceedings the certificate of a registration made before or within five years after first publication of the work shall constitute prima facie evidence of the validity of the copyright and of the facts stated in the certificate. The evidentiary weight to be accorded the certificate of a registration made thereafter shall be within the discretion of the court." (USC Title 17, 410)



> To show actionable copying and therefore satisfy Feist’s second prong, a plaintiff must first prove that the alleged infringer copied plaintiffs copyrighted work as a factual matter; to do this, he or she may either present direct evidence of factual copying or...

In [33]:
print(lotus_majority.holdings[5])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact <Borland International> copied <the Lotus menu command hierarchy> in creating <Quattro's Lotus Emulation Interface>
based on the input
GIVEN: evidence which supports fact <Borland International> copied <the Lotus menu command hierarchy> in creating <Quattro's Lotus Emulation Interface>
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)



> To show actionable copying and therefore satisfy Feist’s second prong, a plaintiff must first prove that the alleged infringer copied plaintiffs copyrighted work as a factual matter; to do this, he or she may either present direct evidence of factual copying or, if that is unavailable, evidence that the alleged infringer had access to the copyrighted work and that the offending and copyrighted works are so similar that the court may infer that there was factual copying (i.e., probative similarity).

In [34]:
print(lotus_majority.holdings[6])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact <Borland International> copied <the Lotus menu command hierarchy> in creating <Quattro's Lotus Emulation Interface>
based on the input
GIVEN: evidence which supports fact <Borland International> had access to <the Lotus menu command hierarchy>
GIVEN: fact <Borland International> published <Quattro's Lotus Emulation Interface>
GIVEN: evidence which supports fact <Quattro's Lotus Emulation Interface> was very similar to <the Lotus menu command hierarchy>
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)



> To show actionable copying and therefore satisfy Feist’s second prong, a plaintiff must first prove that the alleged infringer copied plaintiffs copyrighted work as a factual matter...The plaintiff must then prove that the copying of copyrighted material was so extensive that it rendered the offending and copyrighted works substantially similar.

In [35]:
print(lotus_majority.holdings[7])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact <Borland International> copied constituent elements of <the Lotus menu command hierarchy> that were original
based on the input
GIVEN: fact <Borland International> copied <the Lotus menu command hierarchy> in creating <Quattro's Lotus Emulation Interface>
GIVEN: fact the copying of <Quattro's Lotus Emulation Interface> in <the Lotus menu command hierarchy> was so extensive that it rendered them substantially similar
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)



> Section 102(b) states: “In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illustrated, or embodied in such work.” Because we conclude that the Lotus menu command hierarchy is a method of operation, we do not consider whether it could also be a system, process, or procedure...while original expression is necessary for copyright protection, we do not think that it is alone sufficient. Courts must still inquire whether original expression falls within one of the categories foreclosed from copyright protection by § 102(b), such as being a “method of operation.”

In [36]:
print(lotus_majority.holdings[8])

the rule that the court MUST ALWAYS accept the result
RESULT: fact it is false that <the Lotus menu command hierarchy> was copyrightable
based on the input
GIVEN: fact <the Lotus menu command hierarchy> was a method of operation
DESPITE: fact a text described <the Lotus menu command hierarchy>
DESPITE: fact <the Lotus menu command hierarchy> was an original work
according to the legislation
GIVEN: "In no case does copyright protection for an original work of authorship extend to any" (USC Title 17, 102)
GIVEN: "method of operation" (USC Title 17, 102)



> We hold that the Lotus menu command hierarchy is an uneopyrightable “method of operation.” The Lotus menu command hierarchy provides the means by which users control and operate Lotus 1-2-3. If users wish to copy material, for example, they use the “Copy” command. If users wish to print material, they use the “Print” command. Users must use the command terms to tell the computer what to do. Without the menu command hierarchy, users would not be able to access and control, or indeed make use of, Lotus 1-2-3’s functional capabilities.

In [37]:
print(lotus_majority.holdings[9])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact <the Lotus menu command hierarchy> was a method of operation
based on the input
GIVEN: fact <Lotus 1-2-3> was a computer program
GIVEN: fact <the Lotus menu command hierarchy> provided the means by which users controlled and operated <Lotus 1-2-3>
GIVEN: fact without <the Lotus menu command hierarchy>, users would not have been able to access and control, or indeed make use of, <Lotus 1-2-3>’s functional capabilities
GIVEN: fact for another computer program to by operated in substantially the same way as <Lotus 1-2-3>, the other program would have to copy <the Lotus menu command hierarchy>
DESPITE: fact the developers of <Lotus 1-2-3> made some expressive choices in choosing and arranging the terms in <the Lotus menu command hierarchy>
according to the legislation
GIVEN: "In no case does copyright protection for an original work of authorship extend to any" (USC Title 17, 102)
GIVEN: "method of operation" (USC Title 1

> We do not think that “methods of operation” are limited to abstractions; rather, they are the means by which a user operates something.

In [38]:
print(lotus_majority.holdings[10])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact <the Lotus menu command hierarchy> was a method of operation
based on the input
GIVEN: fact <the Lotus menu command hierarchy> was the means by which a person operated <Lotus 1-2-3>
DESPITE: fact it is false that <the Lotus menu command hierarchy> was an abstraction
according to the legislation
GIVEN: "In no case does copyright protection for an original work of authorship extend to any" (USC Title 17, 102)
GIVEN: "method of operation" (USC Title 17, 102)



> In other words, to offer the same capabilities as Lotus 1-2-3, Borland did not have to copy Lotus’s underlying code (and indeed it did not); to 'allow users to operate its programs in substantially the same way, however, Bor-land had to copy the Lotus menu command hierarchy. Thus the Lotus 1-2-3 code is not a uncopyrightable “method of operation.”

In [39]:
print(lotus_majority.holdings[11])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact it is false that <Lotus 1-2-3> was a method of operation
based on the input
GIVEN: fact <Lotus 1-2-3> was a computer program
GIVEN: fact it is false that the precise formulation of <Lotus 1-2-3>'s code was necessary for it to work
DESPITE: fact computer code was necessary for <Lotus 1-2-3> to work
according to the legislation
GIVEN: "In no case does copyright protection for an original work of authorship extend to any" (USC Title 17, 102)
GIVEN: "method of operation" (USC Title 17, 102)



### *Oracle v. Google* 750 F.3d 1339 (2014)

> By statute, a work must be “original” to qualify for copyright protection. 17 U.S.C. § 102(a).

In [40]:
print(oracle.holdings[0])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact it is false that <the Java API> was copyrightable
based on the input
GIVEN: fact it is false that <the Java API> was an original work
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)



> Original, as the term is used in copyright, means only that the work was independently created by the author (as opposed to copied from other works), and that it possesses at least some minimal degree of creativity.

In [41]:
print(oracle.holdings[1])

the rule that the court MUST ALWAYS accept the result
RESULT: fact <the Java API> was an original work
based on the input
GIVEN: fact <the Java API> was independently created by the author, as opposed to copied from other works
GIVEN: fact <the Java API> possessed at least some minimal degree of creativity
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)



> Copyright protection extends only to the expression of an idea — not to the underlying idea itself...In the Ninth Circuit, while questions regarding originality are considered questions of copyrightability, concepts of merger and scenes a faire are affirmative defenses to claims of infringement.

In [42]:
print(oracle.holdings[2])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact <the Java API> was copyrightable
based on the input
GIVEN: fact <the Java API> was an original work
GIVEN: fact <the Java API> was the expression of an idea
GIVEN: fact it is false that <the Java API> was an idea
DESPITE: fact <the Java API> was essentially the only way to express the idea that it embodied
DESPITE: fact <the Java API> was a scene a faire
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)
and despite the legislation
DESPITE: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regar

> The literal elements of a computer program are the source code and object code.

In [43]:
print(oracle.holdings[3])
print(oracle.holdings[4])

the rule that the court MUST ALWAYS accept the result
RESULT: fact <the Java API> was a literal element of <the Java language>
based on the input
GIVEN: fact <the Java language> was a computer program
GIVEN: fact <the Java API> was the source code of <the Java language>
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)
GIVEN: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illustrated, or embodied in such work." (USC Title 17, 102)

the rule that the court MUST ALWAYS accept the result
RESULT: fact <th

> It is well established that copyright protection can extend to both literal and non-literal elements of a computer program. See Altai 982 F.2d at 702.

In [44]:
print(oracle.holdings[5])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact <the Java API> was copyrightable
based on the input
GIVEN: fact <the Java language> was a computer program
GIVEN: fact <the Java API> was a literal element of <the Java language>
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)
and despite the legislation
DESPITE: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illustrated, or embodied in such work." (USC Title 17, 102)



> The non-literal components of a computer program include, among other things, the program’s sequence, structure, and organization, as well as the program’s user interface.

In [45]:
print(oracle.holdings[6])
print(oracle.holdings[7])

the rule that the court MUST ALWAYS accept the result
RESULT: fact <the Java API> was a non-literal element of <the Java language>
based on the input
GIVEN: fact <the Java language> was a computer program
GIVEN: fact <the Java API> was the sequence, structure, and organization of <the Java language>
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)
GIVEN: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illustrated, or embodied in such work." (USC Title 17, 102)

the rule that the court MUST ALWAYS acce

> It is well established that copyright protection can extend to both literal and non-literal elements of a computer program...As discussed below, whether the non-literal elements of a program “are protected depends on whether, on the particular facts of each case, the component in question qualifies as an expression of an idea, or an idea itself.”

In [46]:
print(oracle.holdings[8])
print(oracle.holdings[9])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact <the Java API> was copyrightable
based on the input
GIVEN: fact <the Java language> was a computer program
GIVEN: fact <the Java API> was a non-literal element of <the Java language>
GIVEN: fact <the Java API> was the expression of an idea
GIVEN: fact it is false that <the Java API> was an idea
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)
and despite the legislation
DESPITE: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illust

> On appeal, Oracle argues that the district court’s reliance on Lotus is misplaced because it is distinguishable on its facts and is inconsistent with Ninth Circuit law. We agree. First, while the defendant in Lotus did not copy any of the underlying code, Google concedes that it copied portions of Oracle’s declaring source code verbatim. Second, the Lotus court found that the commands at issue there (copy, print, etc.) were not creative, but it is undisputed here that the declaring code and the structure and organization of the API packages are both creative and original. Finally, while the court in Lotus found the commands at issue were “essential to operating” the system, it is undisputed that— other than perhaps as to the three core packages — Google did not need to copy the structure, sequence, and organization of the Java API packages to write programs in the Java language.\nMore importantly, however, the Ninth Circuit has not adopted the court’s “method of operation” reasoning in Lotus, and we conclude that it is inconsistent with binding precedent.

In [47]:
print(oracle.holdings[10])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact <the Java API> was copyrightable
based on the input
GIVEN: fact <the Java language> was a computer program
GIVEN: fact <the Java API> was a set of application programming interface declarations
GIVEN: fact <the Java API> was an original work
GIVEN: fact <the Java API> was a non-literal element of <the Java language>
GIVEN: fact <the Java API> was the expression of an idea
GIVEN: fact it is false that <the Java API> was essentially the only way to express the idea that it embodied
GIVEN: fact <the Java API> was creative
GIVEN: fact it was possible to use <the Java language> without copying <the Java API>
DESPITE: fact <the Java API> was a method of operation
DESPITE: fact <the Java API> contained short phrases
DESPITE: fact <the Java API> became so popular that it was the industry standard
DESPITE: fact there was a preexisting community of programmers accustomed to using <the Java API>
according to the legislation
GIV

> In the Ninth Circuit, while questions regarding originality are considered questions of copyrightability, concepts of merger and scenes a faire are affirmative defenses to claims of infringement.

In [48]:
print(oracle.holdings[11])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact it is false that <Google> infringed the copyright on <the Java API>
based on the input
GIVEN: fact <the Java API> was a scene a faire
DESPITE: fact <the Java API> was copyrightable
according to the legislation
GIVEN: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illustrated, or embodied in such work." (USC Title 17, 102)
and despite the legislation
DESPITE: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)



> In the Ninth Circuit, while questions regarding originality are considered questions of copyrightability, concepts of merger and scenes a faire are affirmative defenses to claims of infringement...Under the merger doctrine, a court will not protect a copyrighted work from infringement if the idea contained therein can be expressed in only one way.

In [49]:
print(oracle.holdings[12])
print(oracle.holdings[13])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact it is false that <Google> infringed the copyright on <the Java API>
based on the input
GIVEN: fact <the Java API> was essentially the only way to express the idea that it embodied
DESPITE: fact <the Java API> was copyrightable
according to the legislation
GIVEN: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illustrated, or embodied in such work." (USC Title 17, 102)
and despite the legislation
DESPITE: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)

the rule that 

#### A missing holding

The following text represents a rule posited by the Oracle court, but it's not currently possible to create a corresponding Rule object, because AuthoritySpoke doesn't yet include "Argument" objects.

> Google responds that Oracle waived its right to assert copyrightability based on the 7,000 lines of declaring code by failing “to object to instructions and a verdict form that effectively eliminated that theory from the case.” Appellee Br. 67...We find that Oracle did not waive arguments based on Google’s literal copying of the declaring code.

> Regardless of when the analysis occurs, we conclude that merger does not apply on the record before us...We have recognized, however, applying Ninth Circuit law, that the “unique arrangement of computer program expression ... does not merge with the process so long as alternate expressions are available.”...The evidence showed that Oracle had “unlimited options as to the selection and arrangement of the 7000 lines Google copied.”...This was not a situation where Oracle was selecting among preordained names and phrases to create its packages.

In [50]:
print(oracle.holdings[14])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact it is false that <the Java API> was essentially the only way to express the idea that it embodied
based on the input
GIVEN: fact <Sun Microsystems> created <the Java API>
GIVEN: fact when creating <the Java API>, <Sun Microsystems> could have selected and arranged its names and phrases in unlimited different ways
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)
and despite the legislation
DESPITE: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described

> the relevant question for copyright-ability purposes is not whether the work at issue contains short phrases — as literary works often do — but, rather, whether those phrases are creative.

In [51]:
print(oracle.holdings[15])

the rule that the court MUST SOMETIMES accept the result
RESULT: fact <the Java API> was copyrightable
based on the input
GIVEN: fact <the Java API> was a literary work
GIVEN: fact the short phrases in <the Java API> were creative
DESPITE: fact <the Java API> contained short phrases
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)
and despite the legislation
DESPITE: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illustrated, or embodied in such work." (USC Title 17, 102)
DESPITE: "The following are 

> In the computer context, “the scene a faire doctrine denies protection to program elements that are dictated by external factors such as ‘the mechanical specifications of the computer on which a particular program is intended to run’ or ‘widely accepted programming practices within the computer industry. Like merger, the focus of the scenes a faire doctrine is on the circumstances presented to the creator, not the copier.

In [52]:
print(oracle.holdings[16])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact <the Java API> was a scene a faire
based on the input
GIVEN: fact <the Java language> was a computer program
GIVEN: fact <the Java API> was an element of <the Java language>
GIVEN: fact the creation of <the Java API> was dictated by external factors such as the mechanical specifications of the computer on which <the Java language> was intended to run or widely accepted programming practices within the computer industry
according to the legislation
GIVEN: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illustrated, or embodied in such work." (USC Title 17, 102)
and despite the legislation
DESPITE: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, 

> Specifically, we find that Lotus is inconsistent with Ninth Circuit case law recognizing that the structure, sequence, and organization of a computer program is eligible for copyright protection where it qualifies as an expression of an idea, rather than the idea itself.

In [53]:
print(oracle.holdings[17])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact <the Java API> was copyrightable
based on the input
GIVEN: fact <the Java language> was a computer program
GIVEN: fact <the Java API> was the structure, sequence, and organization of <the Java language>
GIVEN: fact <the Java API> was the expression of an idea
GIVEN: fact it is false that <the Java API> was an idea
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)
and despite the legislation
DESPITE: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described

> an original work — even one that serves a function — is entitled to copyright protection as long as the author had multiple ways to express the underlying idea. Section 102(b) does not, as Google seems to suggest, automatically deny copyright protection to elements of a computer program that are functional.

In [54]:
print(oracle.holdings[18])

the rule that the court MUST ALWAYS accept the result
RESULT: fact <the Java API> was copyrightable
based on the input
GIVEN: fact <the Java API> was an original work
GIVEN: fact <Sun Microsystems> was the author of <the Java API>
GIVEN: fact when creating <the Java API>, <Sun Microsystems> had multiple ways to express its underlying idea
DESPITE: fact <the Java API> served a function
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)
and despite the legislation
DESPITE: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is descri

> Until either the Supreme Court or Congress tells us otherwise, we are bound to respect the Ninth Circuit’s decision to afford software programs protection under the copyright laws. We thus decline any invitation to declare that protection of software programs should be the domain of patent law, and only patent law.

In [55]:
print(oracle.holdings[19])

the rule that the court MAY SOMETIMES accept the result
RESULT: fact <the Java API> was copyrightable
based on the input
GIVEN: fact <the Java language> was a computer program
according to the legislation
GIVEN: "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device." (USC Title 17, 102)
and despite the legislation
DESPITE: "In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illustrated, or embodied in such work." (USC Title 17, 102)

